In [136]:
from pyspark.sql import *
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import pyspark.sql.functions as F
from IPython.display import display
from pyspark.sql.functions import to_timestamp
import pytz
from pyspark.sql.functions import *


In [137]:
spark = SparkSession.builder.appName("test-app").getOrCreate()

In [138]:
df = spark.read.csv("./TestData/oildata.csv", 
                            header="true", 
                            inferSchema="true")
df.createOrReplaceTempView("data")
df.show()


+-------------------+--------+-----+-------------------+----------------------+----------+
|          Timestamp| Well_ID|Stage|Engine_Oil_Pressure|Proppant_Concentration|Fluid_Rate|
+-------------------+--------+-----+-------------------+----------------------+----------+
|2023-06-08 09:00:00|Well-001|    1|                100|                  2000|       500|
|2023-06-08 09:30:00|Well-001|    1|                105|                  2100|       550|
|2023-06-08 10:00:00|Well-001|    1|                102|                  2200|       600|
|2023-06-08 10:30:00|Well-001|    2|                 98|                  2300|       620|
|2023-06-08 11:00:00|Well-001|    2|                102|                  2400|       650|
|2023-06-08 11:30:00|Well-001|    2|                100|                  2500|       680|
|2023-06-08 12:00:00|Well-001|    3|                 95|                  2600|       700|
|2023-06-08 12:30:00|Well-001|    3|                 99|                  2700|       730|

In [139]:
sql = """
SELECT Stage, AVG(Engine_Oil_Pressure) as avg_EOP, AVG(Fluid_Rate) as avg_FR
FROM data
GROUP BY Stage
"""
query = spark.sql(sql)
query.toPandas().sort_values(by=['Stage'])


,Stage,avg_EOP,avg_FR
0,1,100.933333,550.0
3,2,101.166667,650.0
1,3,99.900000,730.0
2,4,100.900000,810.0


In [140]:
result_df=df.select('Engine_Oil_Pressure', 'Proppant_Concentration', 'Fluid_Rate', 'Well_ID','Stage')
result_df=result_df.toPandas()
result_df['Combined_Value'] = (result_df['Engine_Oil_Pressure'] * result_df['Fluid_Rate']) / result_df['Proppant_Concentration'] #moles*psi/liter
result_pivot = result_df.pivot_table(index='Well_ID', columns='Stage', values='Combined_Value', aggfunc='mean')
fig = px.imshow(result_pivot)
display(fig)


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'Stage: %{x}<br>Well_ID: %{y}<br>color: %{z}<extra></extra>',
              'name': '0',
              'type': 'heatmap',
              'x': array([1, 2, 3, 4], dtype=int32),
              'xaxis': 'x',
              'y': array(['Well-001', 'Well-002', 'Well-003', 'Well-004', 'Well-005', 'Well-006',
                          'Well-007', 'Well-008', 'Well-009', 'Well-010'], dtype=object),
              'yaxis': 'y',
              'z': array([[26.77272727, 27.0807971 , 26.76691087, 26.99846496],
                          [26.24963925, 27.53205072, 27.0372541 , 27.26844271],
                          [26.51118326, 27.80285024, 27.30759734, 27.53842047],
                          [25.98809524, 27.26125121, 27.0372541 , 27.26844271],
                          [26.77272727, 27.0807971 , 26.76691087, 26.99846496],
                          [26.24963925, 27.53205072, 27.0372541 , 27.26844271],
                          [26.51118326, 27.80285024, 27.30759734, 27.53842047],
                          [25.98809524, 27.26125121, 27.0372541 , 27.26844271],
                          [26.77272727, 27.0807971 , 26.76691087, 26.99846496],
                          [26.24963925, 27.53205072, 27.0372541 , 27.26844271]])}],
    'layout': {'coloraxis': {'colorscale': [[0.0, '#0d0887'], [0.1111111111111111,
                                            '#46039f'], [0.2222222222222222,
                                            '#7201a8'], [0.3333333333333333,
                                            '#9c179e'], [0.4444444444444444,
                                            '#bd3786'], [0.5555555555555556,
                                            '#d8576b'], [0.6666666666666666,
                                            '#ed7953'], [0.7777777777777778,
                                            '#fb9f3a'], [0.8888888888888888,
                                            '#fdca26'], [1.0, '#f0f921']]},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y',
                         'constrain': 'domain',
                         'domain': [0.0, 1.0],
                         'scaleanchor': 'y',
                         'title': {'text': 'Stage'}},
               'yaxis': {'anchor': 'x',
                         'autorange': 'reversed',
                         'constrain': 'domain',
                         'domain': [0.0, 1.0],
                         'title': {'text': 'Well_ID'}}}
})

In [ ]:
timezone = 'America/New_York'
df = df.withColumn('Timestamp', from_utc_timestamp(df['Timestamp'], lit(timezone)))
df = df.withColumn('Timestamp', to_timestamp('Timestamp', 'yyyy-MM-dd HH:mm:ss'))

# Convert the PySpark DataFrame to a Pandas DataFrame
df_pandas = df.toPandas()

fig = px.scatter(df, x='Timestamp', y='Engine_Oil_Pressure', color='Well_ID', symbol='Stage',
                 title='Engine Oil Pressure vs. Time', labels={'Engine_Oil_Pressure': 'Engine Oil Pressure'})

# Display the plot
fig.show()


AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve "from_utc_timestamp(Timestamp, America/New_York)" due to data type mismatch: Parameter 1 requires the "TIMESTAMP" type, however "Timestamp" has the type "BIGINT".;
'Project [from_utc_timestamp(Timestamp#1795L, America/New_York) AS Timestamp#1804, Well_ID#1709, Stage#1710, Engine_Oil_Pressure#1711, Proppant_Concentration#1712, Fluid_Rate#1713]
+- Project [(cast(cast(Timestamp#1788 as timestamp) as bigint) * cast(1000 as bigint)) AS Timestamp#1795L, Well_ID#1709, Stage#1710, Engine_Oil_Pressure#1711, Proppant_Concentration#1712, Fluid_Rate#1713]
   +- Project [to_timestamp(Timestamp#1708, Some(yyyy-MM-dd HH:mm:ss), TimestampType, Some(America/Chicago), false) AS Timestamp#1788, Well_ID#1709, Stage#1710, Engine_Oil_Pressure#1711, Proppant_Concentration#1712, Fluid_Rate#1713]
      +- Relation [Timestamp#1708,Well_ID#1709,Stage#1710,Engine_Oil_Pressure#1711,Proppant_Concentration#1712,Fluid_Rate#1713] csv


23/06/08 20:53:23 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 7206272 ms exceeds timeout 120000 ms
23/06/08 20:53:24 WARN SparkContext: Killing executors is not supported by current scheduler.
23/06/08 20:53:24 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.